In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import calendar
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from wordcloud import WordCloud, STOPWORDS
import folium
from folium import plugins

# 1. Merge datasets

In [2]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/NUSE 934 611(M) 2017-2018.dsv'
data2018=pd.read_csv(data_location,delimiter="|")

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/NUSE 934-611-611M ENERO2019.csv'
data2019=pd.read_csv(data_location,delimiter=";")

In [4]:
frames = [data2018, data2019]
data = pd.concat(frames)
merged_nuse = data.loc[data['TIPO_DETALLE'] == '934 - RIÑA']
merged_nuse.reset_index(inplace=True)
merged_nuse.drop(columns=['index'], inplace=True)

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
merged_nuse

In [ ]:
merged_nuse.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/merged_nuse.csv',index=None)

# 2. Rebuild missing data

In [5]:
import import_ipynb
import ws_address
from selenium.common.exceptions import TimeoutException

importing Jupyter notebook from ws_address.ipynb


In [6]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [7]:
tags = ["Dirección ingresada: ","Dirección encontrada: ","Tipo dirección: ","Código postal: ","Sector catastral: ",
        "UPZ: ","Localidad: ","Latitud: ","Longitud: ","CHIP: "]
def parse_address_ws(ws_result):
    location = {}
    for idx in range(len(tags)-1):
        location[tags[idx].replace(': ','')] = find_between(ws_result,tags[idx],tags[idx+1])
    return location

In [8]:
def find_char(df, column_name, char=''):
    array_char = np.where(df[column_name] == char)
    if array_char[0].size > 0:
        return array_char[0]
    else:
        return 0

In [ ]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor ''":(merged_nuse == '').sum().values,
              "Celdas con valor ' '":(merged_nuse == ' ').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

In [9]:
df1 = merged_nuse.loc[merged_nuse['COD_UPZ'] == '-']
df2 = merged_nuse.loc[merged_nuse['UPZ'] == '-']
df3 = merged_nuse.loc[merged_nuse['COD_SEC_CATAST'] == '-']
df4 = merged_nuse.loc[merged_nuse['SEC_CATASTRAL'] == '-']
df5 = merged_nuse.loc[merged_nuse['COD_BARRIO'] == '-']
df6 = merged_nuse.loc[merged_nuse['BARRIO'] == '-']

In [10]:
df1.equals(df2) and df1.equals(df3) and df1.equals(df4) and df1.equals(df5) and df1.equals(df6)

True

In [11]:
#Try to rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' through address
df_empty_locations_with_address = df1.loc[df1['STR_DIRECCION_INCIDENTE'] != '-']
list_idx_rebuild = list(df_empty_locations_with_address.index.values)

In [ ]:
df_empty_locations_with_address.head()

In [12]:
url='https://mapas.bogota.gov.co'
driver = ws_address.web_scrap_page(url)

In [13]:
df_prueba = df_empty_locations_with_address.loc[df_empty_locations_with_address['STR_DIRECCION_INCIDENTE'] == 'CL 19C 33 68']
df_prueba

,STR_NUMERO_INTERNO,FECHA,HORA,ANIO,MES,COD_LOCALIDAD,LOCALIDAD,COD_UPZ,UPZ,COD_SEC_CATAST,SEC_CATASTRAL,COD_BARRIO,BARRIO,TIPO_UNICO,TIPO_DETALLE,LATITUD,LONGITUD,STR_DIRECCION_INCIDENTE,ESTADO_INCIDENTE,PERIODO_TS,LOG_TEXT
16456,NU_5_273004172,05-APR-17,2010,2017,4,99,SIN LOCALIZACION,-,-,-,-,-,-,934,934 - RIÑA,-1.0,-1.0,CL 19C 33 68,CERRADO,2017/04,"OUTGOING~56~NUSE~273004172~05042017~201010~3118193510~ORIGIN~ALTA PRIORIDAD~LLAMANTE MANIFIESTA QUE UN SUJT LE ESTA FORMANDO PELEA EN LA VIA PUBLICA. // NO ARMAS NO HERIDOS.~BAJA~,BOGOTA D.C.,PUENTE ARANDA,CUNDINAMARCA,,CL 19C 33,68,,,~-74.0893~4.6222~934~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLORIGIN~PRIORITY~DESCRIPTION~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"


In [14]:
list_prueba = list(df_prueba.index.values)

In [27]:
def assign_upz(original_df,index,UPZ_ws_field):
     pass
    #'UPZ': 'ZONA INDUSTRIAL (UPZ108)'

In [28]:
def rebuild_location(original_df, index, driver):
    address = original_df.at[index,'STR_DIRECCION_INCIDENTE']
    print(address)
    result_ws = ws_address.web_scrap_address(driver,address)
    ws_address.delete_address(driver,address)
    print(result_ws)

    if result_ws != "Not found":
        parsed_result = parse_address_ws(result_ws)
        print(parsed_result)
        original_df.at[index,'LATITUD'] = float(parsed_result['Latitud'])
        original_df.at[index,'LONGITUD'] = float(parsed_result['Longitud'])
        original_df.at[index,'LOCALIDAD'] = parsed_result['Localidad']
        original_df.at[index,'SEC_CATASTRAL'] = parsed_result['Sector catastral']
        assign_upz(original_df,index,parsed_result['UPZ'])
        return "Rebuilt"
    else:
        return "Not processed"


In [29]:
registers_to_process = len(list_prueba)
rebuilt_registers = 0
registers_not_processed = 0
other_condition_counter = 0

for index in list_prueba:
    print(index)
    state = rebuild_location(merged_nuse, index, driver)
    print(state)
    
    if state == "Rebuilt":
        rebuilt_registers += 1
    elif state == "Not processed":
        registers_not_processed += 1
    else:
        other_condition_counter += 1

16456
CL 19C 33 68
Dirección ingresada: CL 19C 33 68Dirección encontrada: CL 19C 33 70Tipo dirección: Aproximación por placaCódigo postal: 111611Sector catastral: CUNDINAMARCAUPZ: ZONA INDUSTRIAL (UPZ108)Localidad: PUENTE ARANDALatitud: 4.62274111100004Longitud: -74.0897361079999CHIP: Consultar CHIP asociados al lote   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'CL 19C 33 68', 'Dirección encontrada': 'CL 19C 33 70', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111611', 'Sector catastral': 'CUNDINAMARCA', 'UPZ': 'ZONA INDUSTRIAL (UPZ108)', 'Localidad': 'PUENTE ARANDA', 'Latitud': '4.62274111100004', 'Longitud': '-74.0897361079999'}
Rebuilt


In [30]:
print(rebuilt_registers)
print(registers_not_processed)
print(other_condition_counter)

1
0
0


In [31]:
merged_nuse.loc[merged_nuse['STR_DIRECCION_INCIDENTE'] == 'CL 19C 33 68']

,STR_NUMERO_INTERNO,FECHA,HORA,ANIO,MES,COD_LOCALIDAD,LOCALIDAD,COD_UPZ,UPZ,COD_SEC_CATAST,SEC_CATASTRAL,COD_BARRIO,BARRIO,TIPO_UNICO,TIPO_DETALLE,LATITUD,LONGITUD,STR_DIRECCION_INCIDENTE,ESTADO_INCIDENTE,PERIODO_TS,LOG_TEXT
16456,NU_5_273004172,05-APR-17,2010,2017,4,99,PUENTE ARANDA,-,-,-,CUNDINAMARCA,-,-,934,934 - RIÑA,4.622741,-74.089736,CL 19C 33 68,CERRADO,2017/04,"OUTGOING~56~NUSE~273004172~05042017~201010~3118193510~ORIGIN~ALTA PRIORIDAD~LLAMANTE MANIFIESTA QUE UN SUJT LE ESTA FORMANDO PELEA EN LA VIA PUBLICA. // NO ARMAS NO HERIDOS.~BAJA~,BOGOTA D.C.,PUENTE ARANDA,CUNDINAMARCA,,CL 19C 33,68,,,~-74.0893~4.6222~934~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLORIGIN~PRIORITY~DESCRIPTION~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"


In [10]:

address = 'CL 14'
result = ws_address.web_scrap_address(driver,address)
print(result)
print(parse_address_ws(result))
ws_address.delete_address(driver,address)

address = 'CL 19C 33 68'
result2 = ws_address.web_scrap_address(driver,address)
print(result2)
print(parse_address_ws(result2))
ws_address.delete_address(driver,address)

address = 'CL 19C ESTE 33 68'
result = ws_address.web_scrap_address(driver,address)
print(result)
print(parse_address_ws(result))
ws_address.delete_address(driver,address)


Not found
{'Dirección ingresada': '', 'Dirección encontrada': '', 'Tipo dirección': '', 'Código postal': '', 'Sector catastral': '', 'UPZ': '', 'Localidad': '', 'Latitud': '', 'Longitud': ''}
Dirección ingresada: CL 19C 33 68Dirección encontrada: CL 19C 33 70Tipo dirección: Aproximación por placaCódigo postal: 111611Sector catastral: CUNDINAMARCAUPZ: ZONA INDUSTRIAL (UPZ108)Localidad: PUENTE ARANDALatitud: 4.62274111100004Longitud: -74.0897361079999CHIP: Consultar CHIP asociados al lote   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'CL 19C 33 68', 'Dirección encontrada': 'CL 19C 33 70', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111611', 'Sector catastral': 'CUNDINAMARCA', 'UPZ': 'ZONA INDUSTRIAL (UPZ108)', 'Localidad': 'PUENTE ARANDA', 'Latitud': '4.62274111100004', 'Longitud': '-74.0897361079999'}
Not found
{'Dirección ingresada': '', 'Dirección encontrada': '', 'Tipo dirección': '', 'Código postal': '', 'Sector catastral': '

In [ ]:
import numpy as np
A=pd.DataFrame(np.random.rand(5,5))

In [ ]:
for index, row in A.iterrows():
    for i in range(len(row)):
        row[0]=0

In [ ]:
A

In [ ]:
A.at[4,1]=0

In [ ]:
A